# Warsztat 2: Lakeflow Pipelines & Databricks Jobs Orchestration

**Cel warsztatu:**
- Poznanie **Lakeflow Pipelines** (nowa nazwa dla Delta Live Tables) - deklaratywnych pipeline'ów
- Implementacja Data Quality z Expectations (warn/drop/fail)
- Konfiguracja Multi-task Databricks Jobs
- Dependencies i parametryzacja między taskami
- Monitoring i alerting dla production workflows

**Czas:** 90 minut

**Zakres tematyczny:**
- Lakeflow pipeline creation - example code
- Expectations setup (warn/drop/fail)
- Multi-task Databricks Jobs
- Dependencies i parametryzacja
- Monitoring i alerting

**Dataset:** Warsztat bazuje na plikach z `dataset/`: `orders_batch.json`, `customers.csv`

**WAŻNE:** 
- **Lakeflow** to nowa nazwa marketingowa dla **Delta Live Tables (DLT)**
- W kodzie używamy modułu `dlt` (API Databricks), ale koncepcyjnie mówimy o "Lakeflow Pipelines"
- Lakeflow Pipelines są uruchamiane jako osobne job'y w UI Databricks, nie bezpośrednio w notebooku

---

## 📚 Inicjalizacja środowiska

In [ ]:
%run ../../00_setup

## 📂 Przygotowanie zmiennych pomocniczych

Definiujemy ścieżki do konkretnych plików z folderu `dataset/`:

In [ ]:
# Definiujemy ścieżki do konkretnych plików z dataset/ (bazując na DATASET_BASE_PATH z 00_setup)
ORDERS_JSON = f"{DATASET_BASE_PATH}/orders/orders_batch.json"
CUSTOMERS_CSV = f"{DATASET_BASE_PATH}/customers/customers.csv"
PRODUCTS_PARQUET = f"{DATASET_BASE_PATH}/products/products.parquet"

# Ścieżki do plików streaming (dla Lakeflow streaming tables)
ORDERS_STREAM_DIR = f"{DATASET_BASE_PATH}/orders/"

print("✓ Zmienne pomocnicze zdefiniowane:")
print(f"  ORDERS_JSON: {ORDERS_JSON}")
print(f"  CUSTOMERS_CSV: {CUSTOMERS_CSV}")
print(f"  PRODUCTS_PARQUET: {PRODUCTS_PARQUET}")
print(f"  ORDERS_STREAM_DIR: {ORDERS_STREAM_DIR}")

## 🎯 Część 1: Lakeflow Pipelines - Podstawy

### Co to jest Lakeflow?

**Lakeflow Pipelines** (poprzednio: Delta Live Tables) to deklaratywny framework Databricks do budowania reliable, production-ready data pipelines.

**Kluczowe koncepcje:**
- **Deklaratywne** - opisujesz CO chcesz osiągnąć, nie JAK to zrobić
- **Streaming & Batch** - jedna składnia dla obu paradygmatów
- **Data Quality** - wbudowane expectations (warn/drop/fail)
- **Auto-scaling** - automatyczne zarządzanie zasobami i klastrami
- **Lineage** - automatyczne śledzenie zależności między tabelami
- **Event Log** - szczegółowy monitoring execution i quality metrics

**Lakeflow vs Tradycyjne Notebooks:**

| Aspekt | Tradycyjne Notebooks | Lakeflow Pipelines |
|--------|---------------------|---------------------|
| **Definicja** | Imperatywna (kroki) | Deklaratywna (rezultat) |
| **Zależności** | Ręczne | Automatyczne (DAG) |
| **Quality** | Custom kod | Wbudowane expectations |
| **Monitoring** | Custom logging | Event log + lineage |
| **Retry** | Ręczna logika | Automatyczne |

### Zadanie 1.1: Lakeflow Pipeline Definition - Bronze Layer

**Instrukcje:**
1. Zdefiniuj Bronze table używając `@dlt.table()` i prawdziwego pliku `orders_batch.json`
2. Dodaj audit metadata (ingest_timestamp, source_file)
3. Użyj streaming read dla incremental processing

**📝 UWAGA:** Ten kod jest **definicją** pipeline - zostanie wykonany przez Lakeflow engine, nie bezpośrednio w tym notebooku.

In [ ]:
# TODO: Zadanie 1.1 - Lakeflow Pipeline Definition (Bronze Layer)
# Ten kod musi być wykonany w Lakeflow pipeline, nie w zwykłym notebooku!

import dlt  # Lakeflow używa modułu 'dlt' (Delta Live Tables API)
from pyspark.sql import functions as F

# ================================================================================
# BRONZE LAYER - Raw Orders from JSON
# ================================================================================

@dlt.table(
    name="____",  # TODO: lakeflow_bronze_orders
    comment="Bronze layer: Raw orders from JSON - immutable landing zone",
    table_properties={
        "quality_layer": "bronze",
        "source_format": "json"
    }
)
def lakeflow_bronze_orders():
    """
    Bronze layer: Load raw orders from orders_batch.json.
    
    TODO:
    1. Uzupełnij nazwę tabeli powyżej
    2. Użyj zmiennej ORDERS_JSON (zdefiniowanej wcześniej)
    3. Uzupełnij format danych (json)
    """
    return (
        spark.read
        .format("____")  # TODO: json
        .option("multiLine", "true")
        .load(____)  # TODO: ORDERS_JSON
        # Audit metadata
        .withColumn("_bronze_ingest_timestamp", F.current_timestamp())
        .withColumn("_bronze_source_file", F.lit(ORDERS_JSON))
    )

# ================================================================================
# SILVER LAYER - Cleaned Orders with Expectations
# ================================================================================

@dlt.table(
    name="lakeflow_silver_orders",
    comment="Silver layer: Cleaned and validated orders with data quality checks",
    table_properties={
        "quality_layer": "____"  # TODO: silver
    }
)
# TODO: Dodaj expectations (przykłady poniżej - odkomentuj i uzupełnij)
# @dlt.expect_or_drop("____", "order_id IS NOT NULL")  # TODO: valid_order_id
# @dlt.expect_or_drop("valid_customer_id", "____ IS NOT NULL")  # TODO: customer_id
# @dlt.expect_or_drop("____", "total_amount > 0")  # TODO: positive_amount
def lakeflow_silver_orders():
    """
    Silver layer: Apply data quality checks and business transformations.
    
    TODO:
    1. Odczytaj dane z Bronze używając dlt.read()
    2. Zastosuj deduplikację
    3. Dodaj transformacje biznesowe (order_date, order_status)
    """
    return (
        dlt.read("____")  # TODO: lakeflow_bronze_orders
        
        # Deduplikacja
        .dropDuplicates(["____"])  # TODO: order_id
        
        # Transformacje dat
        .withColumn("order_date", F.to_date(F.col("____")))  # TODO: order_datetime
        .withColumn("order_year", F.year(F.col("order_datetime")))
        
        # Standaryzacja tekstów
        .withColumn("payment_method", F.upper(F.trim(F.col("____"))))  # TODO: payment_method
        
        # Derived column
        .withColumn("order_status", 
                    F.when(F.col("total_amount") > 0, "____")  # TODO: COMPLETED
                     .otherwise("UNKNOWN"))
        
        # Silver metadata
        .withColumn("_silver_processed_timestamp", F.current_timestamp())
    )

# ================================================================================
# GOLD LAYER - Daily Aggregations
# ================================================================================

@dlt.table(
    name="____",  # TODO: lakeflow_gold_daily_sales
    comment="Gold layer: Daily sales aggregations for BI dashboards"
)
def lakeflow_gold_daily_sales():
    """
    Gold layer: Business-level aggregations.
    
    TODO:
    1. Uzupełnij nazwę tabeli
    2. Agreguj po order_date
    3. Oblicz total_orders, total_revenue, avg_order_value
    """
    return (
        dlt.read("____")  # TODO: lakeflow_silver_orders
        .groupBy("____")  # TODO: order_date
        .agg(
            F.count("order_id").alias("____"),  # TODO: total_orders
            F.sum("____").alias("total_revenue"),  # TODO: total_amount
            F.avg("total_amount").alias("____")  # TODO: avg_order_value
        )
        .withColumn("_gold_created_timestamp", F.current_timestamp())
    )

### Zadanie 1.2: Lakeflow Expectations - Data Quality

**Typy Expectations:**

1. **`@dlt.expect()`** - Loguj naruszenia, ale kontynuuj
2. **`@dlt.expect_or_drop()`** - Odrzuć rekordy nie spełniające warunku
3. **`@dlt.expect_or_fail()`** - Zatrzymaj pipeline przy naruszeniu

**Instrukcje:**
1. Dodaj expectations do Silver table:
   - Email musi zawierać '@'
   - Customer_id > 0
   - Country nie może być NULL
2. Użyj różnych typów expectations (drop, fail, log)

**Przykład:**

In [ ]:
# TODO: Zadanie 1.2 - Expectations examples
# (Ten kod jest przykładem - musi być w Lakeflow pipeline)

@dlt.table(name="silver_orders")
@dlt.____("____", "____ IS NOT NULL")  # expect, valid_order_id, order_id
@dlt.expect_or_drop("____", "____ > 0")  # positive_amount, total_amount
@dlt.expect_or_fail("____", "status IN ('____', '____', '____')")  # valid_status, pending, completed, cancelled
def silver_orders():
    """
    Przykład Silver table z expectations.
    
    TODO:
    1. Uzupełnij expectations powyżej (expect, expect_or_drop, expect_or_fail)
    2. Użyj dlt.read_stream() do czytania z Bronze table
    """
    return (
        dlt.read_stream("____")  # TODO: lakeflow_bronze_orders (lub bronze_orders)
        .select(
            "order_id",
            "customer_id",
            F.to_date(F.col("order_date")).alias("order_date"),
            F.col("total_amount").cast("double"),
            "status"
        )
    )

### Zadanie 1.3: Lakeflow Configuration & Deployment

**Instrukcje (UI Databricks):**

1. Przejdź do **Workflows** → **Delta Live Tables** (nazwa w UI może być nadal DLT)
2. Kliknij **Create Pipeline**
3. Konfiguracja:
   - **Pipeline name**: `workshop_lakeflow_pipeline`
   - **Notebook libraries**: Dodaj notebook z definicją Lakeflow pipeline
   - **Target**: `{CATALOG}.{SCHEMA}`
   - **Storage location**: `/mnt/lakeflow/{user}/`
   - **Pipeline mode**: `Triggered` lub `Continuous`
   - **Cluster mode**: `Fixed size` (1 worker dla testów)
4. Kliknij **Create**
5. Kliknij **Start** aby uruchomić pipeline

**Oczekiwany rezultat:**
- Pipeline utworzony i uruchomiony
- Tabele Bronze/Silver/Gold utworzone
- Data quality metrics widoczne w UI

**Zadanie do wykonania:**
- Stwórz Lakeflow pipeline w UI
- Zrzut ekranu z uruchomionego pipeline
- Sprawdź Event Log dla metryki quality

---

## 🔗 Część 2: Databricks Jobs - Multi-task Orchestration

### Co to są Databricks Jobs?

**Databricks Jobs** - orkiestracja multi-task workflows.

**Typy tasków:**
- **Notebook task** - wykonaj notebook
- **Lakeflow Pipeline task** - uruchom Lakeflow pipeline
- **SQL task** - wykonaj SQL query
- **Python wheel task** - uruchom Python package
- **JAR task** - uruchom Java/Scala

### Zadanie 2.1: Widget Parameters - Parametryzacja notebooków

**Instrukcje:**
1. Użyj `dbutils.widgets` do zdefiniowania parametrów
2. Odbierz parametry z Job execution
3. Użyj parametrów w transformacjach

In [ ]:
# TODO: Zadanie 2.1 - Widget parameters

# Definicja widgets
dbutils.widgets.text("____", "____", "Start Date (YYYY-MM-DD)")  # start_date, 2024-01-01
dbutils.widgets.text("____", "2024-12-31", "End Date (YYYY-MM-DD)")  # end_date
dbutils.widgets.dropdown("environment", "dev", ["dev", "____", "____"], "Environment")  # staging, prod

# Odczyt parametrów
start_date = dbutils.widgets.get("____")  # start_date
end_date = dbutils.widgets.get("end_date")
environment = dbutils.widgets.get("____")  # environment

print(f"=== Job Parameters ===")
print(f"Start Date: {start_date}")
print(f"End Date: {end_date}")
print(f"Environment: {environment}")

# Użyj parametrów w query - wczytaj dane z dataset (używając zmiennej ORDERS_JSON)
orders_df = spark.read.json(ORDERS_JSON)

# Filtruj używając parametrów
filtered_orders = orders_df.filter(
    (F.col("order_date") >= start_date) & 
    (F.col("order_date") <= end_date)
)

# Alternatywnie, jeśli dane już są w tabelach Delta:
# filtered_orders = spark.sql(f"""
#     SELECT * 
#     FROM {CATALOG}.{BRONZE_SCHEMA}.____  -- TODO: bronze_orders
#     WHERE order_date BETWEEN '{start_date}' AND '{____}'  -- end_date
# """)

print(f"Filtered orders: {filtered_orders.count()}")

### Zadanie 2.2: Multi-task Job Configuration

**Instrukcje (UI Databricks):**

1. Przejdź do **Workflows** → **Jobs**
2. Kliknij **Create Job**
3. Dodaj pierwszy task:
   - **Task name**: `ingest_bronze`
   - **Type**: `Notebook`
   - **Source**: Wybierz notebook dla Bronze ingestion
   - **Cluster**: `New job cluster` (1 worker)
   - **Parameters**: `{"start_date": "2024-01-01"}`
4. Dodaj drugi task (depends on: `ingest_bronze`):
   - **Task name**: `transform_silver`
   - **Type**: `Notebook`
   - **Source**: Notebook dla Silver transformations
   - **Depends on**: `ingest_bronze`
5. Dodaj trzeci task (depends on: `transform_silver`):
   - **Task name**: `aggregate_gold`
   - **Type**: `Notebook`
   - **Depends on**: `transform_silver`
6. Skonfiguruj **Schedule**: `Cron: 0 2 * * *` (2 AM daily)
7. Dodaj **Email notifications** na failure
8. Kliknij **Create**

**DAG Structure:**
```
ingest_bronze
      ↓
transform_silver
      ↓
aggregate_gold
```

**Zadanie do wykonania:**
- Utwórz Multi-task Job w UI
- Zdefiniuj dependencies między taskami
- Uruchom Job ręcznie (Run Now)
- Sprawdź Runs history

### Zadanie 2.3: Job Dependencies & Conditional Execution

**Instrukcje:**
1. Dodaj task z warunkiem (If/Else logic):
   - Jeśli Bronze ma > 1000 rekordów → uruchom Silver
   - Jeśli nie → wyślij alert
2. Użyj task values do przekazania danych między taskami

In [ ]:
# TODO: Zadanie 2.3 - Task values (przekazywanie danych)

# Task 1: Sprawdź liczbę rekordów w Bronze table
# Użyj zmiennej BRONZE_SCHEMA z 00_setup
bronze_count = spark.table(f"{CATALOG}.{BRONZE_SCHEMA}.____").count()  # TODO: lakeflow_bronze_orders

print(f"Bronze records: {bronze_count}")

# Ustaw task value (dostępne dla następnych tasków)
dbutils.jobs.taskValues.set(
    key="____",  # TODO: bronze_count
    value=bronze_count
)

# Task 2: Odbierz wartość z poprzedniego taska
# (Ten kod jest przykładem - wykonaj w osobnym tasku w Databricks Jobs)
# received_count = dbutils.jobs.taskValues.get(
#     taskKey="____",  # TODO: ingest_bronze (nazwa poprzedniego taska)
#     key="bronze_count",
#     default=0
# )

# if received_count > ____:  # TODO: 1000
#     print("✓ Sufficient data - proceeding to Silver")
# else:
#     raise Exception(f"Insufficient data: {received_count} records")

### Zadanie 2.4: Job Monitoring & Alerting

**Instrukcje (UI Databricks):**

1. W Job configuration, przejdź do **Alerts**
2. Dodaj email notification:
   - **On failure**: `your-email@example.com`
   - **On success**: (opcjonalnie)
3. Skonfiguruj **Retries**:
   - **Max retries**: 3
   - **Timeout**: 3600 seconds (1 hour)
4. Dodaj **SLA**:
   - Jeśli Job trwa > 2 godziny → alert

**Monitoring w kodzie:**

In [ ]:
# TODO: Zadanie 2.4 - Job monitoring code

import time

# Start timer
start_time = time.time()

try:
    # Twój kod transformacji
    print("Processing data...")
    time.sleep(2)  # Symulacja przetwarzania
    
    # Zapisz metrics
    end_time = time.time()
    duration = end_time - start_time
    
    dbutils.jobs.taskValues.set("____", duration)  # execution_time
    
    print(f"✓ Success! Duration: {duration:.2f}s")
    
except Exception as e:
    # Log error
    print(f"✗ Error: {str(e)}")
    
    # Ustaw failure reason
    dbutils.jobs.taskValues.set("failure_reason", str(e))
    
    # Re-raise dla Job failure
    raise

---

## ✅ Podsumowanie warsztatu

**Zrealizowane cele:**
- ✅ Lakeflow Pipelines - deklaratywne pipeline definition
- ✅ Data Quality z Expectations (drop, fail, log)
- ✅ Multi-task Databricks Jobs z dependencies
- ✅ Parametryzacja i task values
- ✅ Monitoring i alerting

**Kluczowe wnioski:**

1. **Lakeflow vs Traditional Spark:**
   - Lakeflow: Deklaratywne, auto-scaling, built-in quality
   - Spark: Imperatywne, manual scaling, custom quality

2. **Kiedy używać Lakeflow Pipelines:**
   - Production data pipelines
   - Data quality critical
   - Need auto-scaling
   - Want automatic lineage

3. **Databricks Jobs Best Practices:**
   - Małe, reusable notebooki (single responsibility)
   - Parametryzacja przez widgets
   - Task dependencies dla DAG
   - Monitoring i alerting dla critical jobs

**Quick Reference:**

| Feature | Lakeflow Pipelines | Traditional Spark |
|---------|---------------------|------------------|
| Składnia | Deklaratywna | Imperatywna |
| Data Quality | Built-in expectations | Custom code |
| Lineage | Automatic | Manual |
| Orchestration | Automatic | Manual (Jobs) |
| Scaling | Auto | Manual |

**Następne kroki:**
- **Kolejny warsztat**: 03_governance_integrations_workshop.ipynb
- **Dokumentacja**: [Lakeflow Pipelines Guide](https://docs.databricks.com/delta-live-tables/)
- **Best Practices**: [Databricks Jobs](https://docs.databricks.com/workflows/jobs/)

---

## 🧹 Cleanup (opcjonalnie)

In [ ]:
# Usuń widgets
# dbutils.widgets.removeAll()

# Wyczyść cache
# spark.catalog.clearCache()